# RNN Model Development

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Masking, GRU
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from datetime import datetime, timedelta


# GBQ importing data

In [65]:
list_of_dfs = ["ban", "cummies", "dinu", "doge",
"doggy", "elon", "erc20", "ftm", "grlc", "hoge",
"lowb", "mona", "samo", "shib", "shibx", "smi",
"wow", "yooshi","yummy"]

project_id = 'crypto-prediction-333213'

def get_data():
    
    dfs = []

    for df in list_of_dfs:

        sql = f"""
        SELECT *
        FROM `crypto-prediction-333213.crypto_BQDB.{df}`;
        """

        dfs.append(pd.read_gbq(sql, project_id=project_id, dialect='standard'))
        
    return dfs

def data_cleaning(dfs):
    
    # takes as input the list of dataframes

    dfs_new = []

    for i in range(0,len(dfs)):
        df = dfs[i]
        price_col = [i for i in df.columns if "price" in i][0]
        
        # data cleaning
        df = df.sort_values(by="datetime")
        df = df.drop_duplicates(keep= "last")
        df = df.set_index("datetime")
        df[price_col].interpolate(method="linear", inplace= True)
        
        # adding financial indicators
        df["hourly_pct_change"] = df[price_col].pct_change(1)
        df["MA12_hours"] = df[price_col].rolling(window=12).mean()
        df["MA72_hours"] = df[price_col].rolling(window=72).mean()
        df["MAshort_over_long"] = df["MA12_hours"] > df["MA72_hours"]
        df["MAshort_over_long"] = df["MAshort_over_long"].apply(lambda x: 1 if x == True else 0)
        
        cols = [i for i in df.columns[2:]]
        for col in cols:
            df[col] = df[col].fillna(-99)
        
        dfs_new.append(df)
    
    return dfs_new



In [62]:
dfs = get_data()
dfs[0]

,datetime,price_ban,tweets_ban
0,2021-09-09 05:00:00+00:00,0.008681,0
1,2021-09-09 06:00:00+00:00,0.008569,0
2,2021-09-09 11:00:00+00:00,0.008541,0
3,2021-09-09 18:00:00+00:00,0.008598,0
4,2021-09-10 04:00:00+00:00,0.008543,0
...,...,...,...
1995,2021-10-23 17:00:00+00:00,0.008740,61
1996,2021-09-25 19:00:00+00:00,0.007421,70
1997,2021-10-02 17:00:00+00:00,0.007332,73
1998,2021-11-11 15:00:00+00:00,0.022351,79


In [66]:
dfs = get_data()
dfs = data_cleaning(dfs)

In [49]:
dfs[0]

,price_ban,tweets_ban,hourly_pct_change,MA12_hours,MA72_hours,MAshort_over_long
datetime,,,,,,
2021-09-09 01:00:00+00:00,-4.747466,1,-99.000000,-99.000000,-99.000000,0
2021-09-09 02:00:00+00:00,-4.759288,4,-0.011752,-99.000000,-99.000000,0
2021-09-09 03:00:00+00:00,-4.764473,2,-0.005172,-99.000000,-99.000000,0
2021-09-09 04:00:00+00:00,-4.751101,2,0.013462,-99.000000,-99.000000,0
2021-09-09 05:00:00+00:00,-4.746628,0,0.004483,-99.000000,-99.000000,0
...,...,...,...,...,...,...
2021-12-01 04:00:00+00:00,-4.070548,2,0.012725,0.016840,0.016611,1
2021-12-01 05:00:00+00:00,-4.077048,0,-0.006480,0.016864,0.016637,1
2021-12-01 06:00:00+00:00,-4.080204,0,-0.003151,0.016880,0.016662,1


# Reshaping Data

In [5]:
horizon = 24

history_size = 48

def get_X_y(history_size, horizon, dataset):

    X = []
    y = []

    for i in range(0, dataset.shape[0]-history_size-horizon):
        X.append(dataset[i:i+history_size,:])
        y.append(dataset[i+history_size: i+history_size+horizon,0]) # should be series of next 24 values for 24h forecast

    return np.array(X), np.array(y)

def reshape_data(dfs, history_size):

    # scaling the data & get_X_y for all datasets
    X = []
    y = []
    scalers = []

    for df in dfs:

        # log transform
        price_col = [i for i in df.columns if "price" in i][0]
        df[price_col] = np.log(df[price_col])
        
        data = df.values
        
        # scaling
        scaler = MinMaxScaler()
        
        scaler.fit(data)
        data = scaler.transform(data)
        
        # get X and y
        X_arr, y_arr = get_X_y(history_size, horizon, data)
        
        X.append(X_arr)
        y.append(y_arr)
        scalers.append(scaler)

    # stacking data three-dimensionally
    X = np.stack(X, axis= 2).reshape(len(X[0]),history_size,-1)
    y = np.stack(y, axis = 2).reshape(len(y[0]),-1)

    return X, y, scalers


In [6]:
dfs_tmp = dfs[:]

In [7]:
X, y, scalers =reshape_data(dfs_tmp, history_size)

In [8]:
X.shape

(1928, 48, 114)

In [9]:
y.shape

(1928, 456)

In [10]:
def garrit(coins, horizon):    
    number_of_cryptos = 2
    # instantiating a model
    model = Sequential()

    # first network layer
    model.add(Masking(mask_value=-1))
    model.add(LSTM(units = 50, activation= "tanh", return_sequences= True))
    model.add(Dropout(0.2))

    # network layer's 2 - 5
    model.add(LSTM(units= 50, activation= "tanh", return_sequences= True))
    model.add(LSTM(units= 10, activation= "tanh", return_sequences= False))
    model.add(Dropout(0.2))

    # network output layer
    # predicting future_horizon = 24 hours
    # for two cryptos
    model.add(Dense(coins*horizon, activation= "linear")) # should be 24 for 24h forecast

    model.compile(optimizer= "adam", loss= "mse")

    return model
    #XXXXX

# Modeling

In [105]:
def morten(coins, horizon):
    coins = 19
    # instantiating a model
    model = Sequential()

    # first network layer
    model.add(Masking(mask_value=-99))
    model.add(GRU(units = 50, activation= "tanh", return_sequences= True))
    model.add(Dropout(0.2))

    # network layer's 2 - 5
    model.add(GRU(units= 50, activation= "tanh", return_sequences= True))
    model.add(Dropout(0.2))
    model.add(GRU(units= 25, activation= "tanh", return_sequences= False))
    model.add(Dropout(0.2))

    # network output layer
    # predicting future_horizon = 24 hours
    model.add(Dense(coins*horizon, activation= "linear")) # should be 24 for 24h forecast

    model.compile(optimizer= "adam", loss= "mse")

    return model

In [106]:
model = morten(19,24)

In [13]:
model

In [107]:
es = EarlyStopping(patience = 10, restore_best_weights= True)

model.fit(X, 
          y,
          validation_split= 0.2,
          epochs = 50,
          batch_size= 64,
          callbacks= [es],
          verbose= 1)

Epoch 1/50
25/25 [==============================] - 21s 287ms/step - loss: 0.2086 - val_loss: 0.1164
Epoch 2/50
25/25 [==============================] - 3s 122ms/step - loss: 0.0798 - val_loss: 0.0513
Epoch 3/50
25/25 [==============================] - 3s 125ms/step - loss: 0.0510 - val_loss: 0.0413
Epoch 4/50
25/25 [==============================] - 3s 122ms/step - loss: 0.0405 - val_loss: 0.0337
Epoch 5/50
25/25 [==============================] - 3s 122ms/step - loss: 0.0326 - val_loss: 0.0302
Epoch 6/50
25/25 [==============================] - 3s 120ms/step - loss: 0.0280 - val_loss: 0.0281
Epoch 7/50
25/25 [==============================] - 3s 120ms/step - loss: 0.0258 - val_loss: 0.0296
Epoch 8/50
25/25 [==============================] - 3s 121ms/step - loss: 0.0242 - val_loss: 0.0293
Epoch 9/50
25/25 [==============================] - 3s 121ms/step - loss: 0.0229 - val_loss: 0.0297
Epoch 10/50
25/25 [==============================] - 3s 120ms/step - loss: 0.0230 - val_loss: 0.031

Baseline Model

In [63]:
test_copy = dfs[0].copy()
for i in range(1,25):
  test_copy[i] = test_copy['price_ban'].shift(i)
test_copy

,datetime,price_ban,tweets_ban,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,2021-09-09 05:00:00+00:00,0.008681,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-09-09 06:00:00+00:00,0.008569,0,0.008681,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-09-09 11:00:00+00:00,0.008541,0,0.008569,0.008681,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-09-09 18:00:00+00:00,0.008598,0,0.008541,0.008569,0.008681,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-09-10 04:00:00+00:00,0.008543,0,0.008598,0.008541,0.008569,0.008681,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,2021-10-23 17:00:00+00:00,0.008740,61,0.007345,0.022252,0.022428,0.007292,0.007196,0.034201,0.007458,0.008846,0.025749,0.007199,0.023465,0.007107,0.007186,0.008708,0.008819,0.007145,0.022400,0.033496,0.007169,0.007339,0.020707,0.025143,0.007274,0.032580
1996,2021-09-25 19:00:00+00:00,0.007421,70,0.008740,0.007345,0.022252,0.022428,0.007292,0.007196,0.034201,0.007458,0.008846,0.025749,0.007199,0.023465,0.007107,0.007186,0.008708,0.008819,0.007145,0.022400,0.033496,0.007169,0.007339,0.020707,0.025143,0.007274
1997,2021-10-02 17:00:00+00:00,0.007332,73,0.007421,0.008740,0.007345,0.022252,0.022428,0.007292,0.007196,0.034201,0.007458,0.008846,0.025749,0.007199,0.023465,0.007107,0.007186,0.008708,0.008819,0.007145,0.022400,0.033496,0.007169,0.007339,0.020707,0.025143
1998,2021-11-11 15:00:00+00:00,0.022351,79,0.007332,0.007421,0.008740,0.007345,0.022252,0.022428,0.007292,0.007196,0.034201,0.007458,0.008846,0.025749,0.007199,0.023465,0.007107,0.007186,0.008708,0.008819,0.007145,0.022400,0.033496,0.007169,0.007339,0.020707


In [14]:
tmp = np.expand_dims(X[0], axis= 0)
tmp.shape

(1, 48, 114)

In [15]:
coins= 19

In [16]:
pred = model.predict(tmp).reshape(horizon, coins)
pred

array([[ 0.31553674,  0.21041544,  0.7923707 ,  0.28320605,  0.396002  ,
         0.47127327,  0.37066314,  0.5543442 ,  0.55429065,  0.42577204,
         0.46080685,  0.36787444,  0.4856592 ,  0.47130674,  0.5204243 ,
         0.34622315,  0.39906946,  0.27701303,  0.36052245],
       [ 0.30914712,  0.22949186,  0.50520265,  0.44615734,  0.34465563,
         0.47320473,  0.25999597,  0.54256266,  0.41961002,  0.4460569 ,
         0.46280077,  0.20529649,  0.47121486,  0.5111708 ,  0.48269647,
         0.41757593,  0.48477003,  0.3897669 ,  0.55606735],
       [ 0.22259349,  0.21415877,  0.5276266 ,  0.38833866,  0.51240504,
         0.34788808,  0.3572773 ,  0.52313423,  0.2872548 ,  0.30979413,
         0.3244713 ,  0.37327462,  0.28110683,  0.33976188,  0.50581944,
         0.36323744,  0.60346234,  0.29552832,  0.7177497 ],
       [ 0.30582628,  0.37190184,  0.49335513,  0.25984845,  0.3298521 ,
         0.3642701 ,  0.2569003 ,  0.38586026,  0.4872382 ,  0.43772966,
         0.402

In [17]:
scalers

[MinMaxScaler(),
 MinMaxScaler(),
 MinMaxScaler(),
 MinMaxScaler(),
 MinMaxScaler(),
 MinMaxScaler(),
 MinMaxScaler(),
 MinMaxScaler(),
 MinMaxScaler(),
 MinMaxScaler(),
 MinMaxScaler(),
 MinMaxScaler(),
 MinMaxScaler(),
 MinMaxScaler(),
 MinMaxScaler(),
 MinMaxScaler(),
 MinMaxScaler(),
 MinMaxScaler(),
 MinMaxScaler()]

In [18]:
df = pd.DataFrame(pred)

# Inverse Transform

In [1]:
def inverse_transformer(y, scaler):

  # inverse scaling
  y = np.c_[y, 1,1,1,1,1]
  y = scaler.inverse_transform(y)
  y= y[:,0]
  
  # inverse log-transforming
  y = np.exp(y)[0]
  return y

def reshape_predictions(prediction_dataframe, list_of_scalers, list_of_dfs):

  for i in range(0,prediction_dataframe.shape[1]):
  
    scaler = scalers[i]
    prediction_dataframe[i] = prediction_dataframe[i].apply(lambda x: inverse_transformer(x,scaler))
    
    # renaming columns
    prediction_dataframe.columns = list_of_dfs

In [ ]:
real_stock_price_doge = inverse_transformer(y_pad_test[0],scaler_doge)
real_stock_price_samo = inverse_transformer(y_pad_test[1],scaler_samo)


predicted_stock_price = inverse_transformer(model.predict(X_pad_test), scaler_doge)

In [ ]:
np.exp(real_stock_price_doge)

# Inverse Log Transform

In [ ]:
real_stock_price_doge = np.exp(real_stock_price_doge)
real_stock_price_samo = np.exp(real_stock_price_samo)

#predicted_stock_price = np.exp(predicted_stock_price)

#predicted_stock_price.shape

# Evaluate

In [ ]:
rmse = np.sqrt(mean_squared_error(real_stock_price, predicted_stock_price))

print(f"RMSE = {rmse}")

plt.figure(figsize= (15,7))
plt.plot(real_stock_price, color = 'blue', label = "Real Price")
plt.plot(predicted_stock_price, color = 'red', label = "Predicted Price")
plt.title("Dogecoin Price Prediction")
plt.xlabel("Time")
plt.ylabel("Dogecoin Price")
plt.legend()

plt.show()

# Notes on different model performances

In [ ]:
eval_dict = {"optimizer": ["adam","adam","adam","adam","adam","adam","adam","adam","adam", "adam", "rmsprop", "rmsprop", "adam"],
             "loss": ["mse","mse","mse","mse","mse","mse","mse","mse","mse", "mse", "mse", "mse", "mse"],
              "history_size": [14, 14, 3, 3, 2, 1, 2, 2, 2, 2, 2, 2, 2],
             "epochs": [500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500],
             "layer0": [50,50,50,50,50,50, 50, 50, 50, 50, 50, 100, 100],
            "layer1": [10, 20, 20, 20, 50, 50, 0, 50, 20, 20, 50, 10, 10],
           "layer2": [10, 10, 10, 20, 10, 10, 0, 50, 0, 10, 10, 0, 0],
           "layer3": [0, 0, 0, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0],
           "layer4": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
           "layer5": [0, 0, 0, 0, 0, 0,0, 0, 0, 0, 0, 0, 0],
            "output": [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
             "batch_size": [32,32,32,32,32,32,32,32,32,32,32,32,32],
           "rmse": [0.051, 0.037, 0.017, 0.018, 0.014, 0.021, 0.020, 0.023, np.NaN, 0.023, np.NaN, 0.020, 0.022]}

eval_df = pd.DataFrame.from_dict(eval_dict)

#for k,v in eval_dict.items():
  #print(f"{k} : {len(v)}")

eval_df

In [ ]:
df_final.head()

In [ ]:
df_pred = pd.DataFrame(X_test[0], columns= ["high", "Google_trends"])
df_pred

In [ ]:
def preprocess(df, scaler):
    """method that pre-process the data"""

    # log transforming the data
    df["high"] = np.log(df["high"])

    # selecting relevant column from df
    dataset = df.values

    # scaling the data
    dataset_scaled = scaler.fit_transform(dataset)

    dataset_scaled = dataset_scaled.reshape(1,dataset.shape[0],dataset.shape[1])

    return dataset_scaled, scaler


def inverse_scaler(pred, scaler):

  pred = inverse_transformer(pred)

  pred = np.exp(pred)

  return pred

In [ ]:
df_pred_preproc, scaler = preprocess(df_pred, scaler)

In [ ]:
pred = model.predict(df_pred_preproc)
prediction = inverse_scaler(pred, scaler)[0][0]

prediction

In [ ]:
df_pred_preproc.shape

In [ ]:
pred = model.predict(df_pred_preproc)[0][0]
pred

# Features

- historical price data
- google price data
- rendite (calculate daily return)
- kumulierte rendite (z.b. letzte 5 Tage)
- short term moving average (letzte 10 Tage)
- long term moving average (letzte 30 Tage)
- alternativ: binäres feature (liegt kurzfristiger MA über langfristigen MA)
- talib (financial indicators inkludieren)
- talib RSI (relative strength index)
- volumen inkludieren

- evtl. auf Klassifikationsproblem umsteigen